---

## Universidad de Costa Rica
### Escuela de Ingeniería Eléctrica
#### IE0405 - Modelos Probabilísticos de Señales y Sistemas

Segundo semestre del 2020

---

* Estudiante: **Nombre completo**
* Carné: **B12345**
* Grupo: **1**


# `P3` - *La demanda energética de electricidad en Costa Rica, parte II*

> Esta actividad extiende el análisis y modelado realizados anteriormente sobre la demanda eléctrica del país a partir de una base de datos del Instituto Costarricense de Electricidad del 2019. El énfasis ahora es encontrar las relaciones que existen entre los modelos de cada hora analizada.

---
* Elaboración de nota teórica y demostración: **Jeaustin Sirias Chacón**, como parte de IE0499 - Proyecto Eléctrico: *Estudio y simulación de aplicaciones de la teoría de probabilidad en la ingeniería eléctrica*.
* Revisión: **Fabián Abarca Calderón**


---
## 1. - Introducción

Ampliando el análisis del Proyecto 2, ahora hacemos referencia a un **vector aleatorio** que contiene una familia de las variables aleatorias en cuestión. Es usual expresar este hecho como:

$$
\vec{X} = (X_1, X_2, X_3, \ldots, X_n)
$$

Hablar de un conjunto multidimensional $\vec{X}$ tiene ciertas implicaciones. Cuando un vector aleatorio tiene dos variables aleatorias es posible darle una interpretación con un **sentido espacial**. Por ejemplo, supóngase que hay cierto vector aleatorio $\vec{V} = (X, Y)$ de dos dimensiones (puesto que, en efecto, hay dos variables aleatorias en $\vec{V}$). Hacer referecia ahora a un "plano" tiene sentido, en especial para evidenciar que cualquier par ordenado $(x_i, y_j)$ representa un punto aleatorio en el plano $XY$. 

Ahora bien, ¿qué ocurre cuando $\vec{V}$ es $n$-dimensional?, ¿podría ser sencillo de manipular cinco o siete dimensiones, o es siquiera posible? Este proyecto programado se abordará desde un enfoque práctico para conocer las relaciones que existen en los datos de demanda energética.

---
## 2. - La librería `fitter` para la determinación de modelos probabilísticos

Hasta el momento en el curso se han encontrado los parámetros de mejor ajuste entre un conjunto de datos (una "muestra") y un modelo probabilístico particular, quizá elegido *arbitrariamente* o por un desarrollado sentido de la intuición, del tipo: "hmm, esa distribución me parece exponencial". Sin embargo, está claro que algunos modelos hacen una mejor descripción de los datos que otros, y no siempre se puede encontrar "a puro ojo". 

En el proyecto anterior se construyó una funcion programada específicamente para hallar el modelo de mejor ajuste para una población específica de muestras, para evidenciar una aplicación práctica de las variables aleatorias. Unas pocas distribuciones eran sometidas a prueba (distribución normal, gamma, beta, Burr tipo XII, etc.) como candidatas al mejor modelo de ajuste; no obstante, el módulo `stats` de la biblioteca SciPy ofrece hasta 80 distribuciones distintas, y sería algo tedioso probarlas todas (y hasta brusco en términos del tiempo de ejecución de Python). 

Para ello la biblioteca [`fitter`](https://fitter.readthedocs.io/en/latest/) será introducida en esta ocasión. Tal cual su nombre lo indica (*ajustador*, en español), esta herramienta provee módulos dedicados a probar todas las distribuciones de `stats`, concluyendo con la mejor distribución (según algún criterio) y sus parámetros de ajuste. Los métodos a emplear del módulo `fitter` son los siguientes:

```python
from fitter import Fitter # Invocación de la clase Fitter

f = Fitter()              # Para instanciar la clase
f.fit()                   # Para evaluar las 80+ distribuciones
f.get_best()              # Para hallar el mejor modelo 
```

### 2.1. - Instalación de `fitter`

Puede verificar si la librería está instalada en su computadora revisando la lista de paquetes de Python, en la terminal

```bash
pip list
```

Si no está, se puede instalar con

```bash
pip install fitter
```

---
## 3. - Funciones auxiliares

Es útil implementar ciertas funciones que simplifiquen las asignaciones a resolver, no obstante, se observará que algunas de ellas "heredan" características e instrucciones que contenían las funciones en el anterior proyecto. 

### 3.1. - Funciones implementadas

Se contará con las siguientes funciones:

1. `extraer_datos(archivo_json, hora)`: Importa la base de datos completa y devuelve los datos de potencia a la hora indicada en un *array* de valores. Esta función es idéntica a la del proyecto anterior.

* `distribucion_conjunta(X, Y, bins)`: Pide por parámetros dos variables aleatorias, $X$ y $Y$, y el número de divisones (`bins`) en el histograma. Visualiza un histograma bivariado y retorna los datos de la función bivariada discreta en dos formatos distintos.

* `evaluar_modelos(datos)`: Encuentra la distribución de mejor ajuste y retorna los parámetros del modelo correspondiente para la muestra en `datos`.

* `densidad_marginal()`: Encuentra la función de densidad marginal de una distribución bivariada haciendo una suma a lo largo de un eje, luego grafica contra el modelo de mejor ajuste.

* `energia_diaria(archivo_json)`: Importa la base de datos completa en formato **JSON** y calcula la energía diaria usando [la regla del trapecio](https://es.wikipedia.org/wiki/Regla_del_trapecio) y retorna un vector con el valor de energía de cada día.

* `parametros_energia(vector_energia)`: Recibe por parámetro un vector de energía diaria y retorna la media y la desviación estándar basadas en las muestras ingresadas.

Las llamadas a bibliotecas de Python de interés para este proyecto programado son:

```python
# Para manipular datos de interés y calcular resultados
import numpy as np 

# Para evaluar el modelo de mejor ajuste en una población
from fitter import Fitter 

# Para manipular y leer la base de datos de demanda
import pandas as pd  

# La música de la fiesta
from scipy import stats    

# Para visualizar resultados relevantes
import matplotlib.pyplot as plt  

# Para manipulación de formatos temporales
from datetime import datetime    
```

#### 3.1.1. - Importación y acondicionamiento con `extraer_datos`

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

def extraer_datos(archivo_json, hora):
    '''Importa la base de datos completa y devuelve los
    datos de potencia a la hora indicada en un
    array de valores.
    '''
    
    # Cargar el "DataFrame"
    df = pd.read_json(archivo_json) 
    
    # Convertir en un array de NumPy
    datos = np.array(df)                

    # Crear vector con los valores demanda en una hora
    demanda = []

    # Extraer la demanda diaria en la hora seleccionada
    for i in range(len(datos)):
        instante = datetime.fromisoformat(datos[i][0]['fechaHora'])
        if instante.hour == hora:
            demanda.append(datos[i][0]['MW'])

    return demanda

#### 3.1.2. - Parámetros y modelos de mejor ajuste con `distribucion_conjunta`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def distribucion_conjunta(X, Y, bins):
    '''Pide por parámetros dos variables aleatorias
    X y Y, así como el número de 'bins' o divisiones
    a emplear para construir el histograma bivariado.
    Crea una gráfica y retorna dos tablas de datos de
    pares ordenados x, y y su probabilidad p asociada.
    '''
    
    np.seterr(all='ignore') # ignorar advertencias
  
    # Se inicializa la figura interactiva 3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Se obtiene el plano de probabilidades para graficar el hist3D
    hist, xbins, ybins = np.histogram2d(X, Y, bins=bins, normed=False)
    hist = hist / sum(sum(hist)) # normalización
    xbins = (xbins + np.roll(xbins, -1))[:-1] / 2.0 
    ybins = (ybins + np.roll(ybins, -1))[:-1] / 2.0 
    
    # Formatos de retorno para la funcion de densidad bivariada discreta
    xyp = [[xbins[i], ybins[j], hist[i][j]] for i in range(bins) for j in range(bins)]
    xy = hist 

    # Se construyen los arreglos para el ancho de bins * bins barras
    xpos, ypos = np.meshgrid(xbins, ybins, indexing="ij")
    xpos = xpos.ravel()
    ypos = ypos.ravel()
    zpos = 0

    # Se dimensiona el ancho visual de las barras (como un sólido).
    dx = dy = 30 * np.ones_like(zpos)
    dz = hist.ravel() 

    # Se visualiza el histograma 3D
    ax.bar3d(xpos, ypos, zpos, dx, dy, dz, zsort='average')
    ax.set_xlabel('La hora 1, $X$')
    ax.set_ylabel('La hora 2, $Y$')
    ax.set_zlabel('Probabilidad')
    plt.show()

    return xyp, xy, xbins, ybins
    

Obsérvese que la función anterior retorna los datos de la función bivariada sobre la base de las variables aleatorias $X$ y $Y$ en dos arreglos distintos `xyp` y `xy` con el objetivo de simplificar la manipulación de datos en las actividades por realizar. Lucen de la siguiente forma:

**El formato `xyp`**

```python
    x   y      P
0  x1  y1  0.001
1  x1  y2  0.020
2  x1  y3  0.092
3  x2  y1  0.093
4  x2  y2  0.083
.   .   .    .
n  xn  yn   Pn
```

**El formato `xy`**

```python
      y1      y2      y3      y4    .   yn
x1  0.030  0.0400  0.0400  0.0400   .    .
x2  0.090  0.0056  0.0400  0.0860   .    .
x3  0.005  0.0450  0.0690  0.0010   .    .
x4  0.060  0.0230  0.0025  0.0014   .    .
.     .       .       .       .     .    .
xn    .       .       .       .     .    .
```

En ambos casos, independientemente, se puede interpretar los elementos $x_n$ y $y_n$ (de las variables `xbins` y `ybins`) como los puntos medios de las divisiones por dimensión (`bins`). Para este caso ambas dimensiones tendrán siempre la misma cantidad de divisiones, $n$, por la forma en que se ha programado la función `distribucion_conjunta`.

#### 3.1.3. - Determinación del modelo de mejor ajuste con `evaluar_modelos`

In [ ]:
from fitter import Fitter

def evaluar_modelos(datos):
    '''Evalúa las 80 distribuciones del módulo stats
    y obtiene el el modelo de mejor ajuste para
    la demanda anual de una hora específica. Retorna
    el nombre de la mejor distribución y una tupla con
    los parámetros del modelo.
    '''
    
    np.seterr(all='ignore') # ignorar advertencias
    
    # Hallar el mejor ajuste con Fitter
    f = Fitter(datos, timeout=120)
    f.fit()
    ajuste = f.get_best()
    
    for i in ajuste.keys():
        dist, params = i, ajuste[i]

    print('------------\nDistribución\n------------')
    print(dist, '\n')
    print('----------\nParámetros\n----------')
    print(params)

    return dist, params

#### 3.1.4. - Comparación de la densidad marginal y un modelo con `densidad_marginal`

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

def densidad_marginal(xy, bins, dist, params, eje):
    '''Encuentra la función de densidad marginal de una
    distribución bivariada haciendo una suma a lo largo de
    un eje, luego grafica contra el modelo de mejor ajuste.
    Se elige eje='x' o eje='y', según sea el caso para la 
    densidad marginal en Y o en X. El parámetro 'xy' es la
    tabla de datos, 'bins' es el vector de valores de potencia 
    xbins o ybins. Los parámetros 'dist' y 'params' corresponden
    al modelo de mejor ajuste retornado por la librería fitter.
    '''
    
    np.seterr(all='ignore') # ignorar advertencias
    
    # Hallar la densidad marginal de X o Y, según se indique en 'eje'
    if eje == 'x':
        filas = len(xy)
        marginal = [sum(xy[i]) for i in range(filas)]
    elif eje == 'y':
        xy = xy.transpose()
        filas = len(xy)
        marginal = [sum(xy[i]) for i in range(filas)]

    # Visualizar modelo de mejor ajuste
    distro = getattr(stats, dist) 
    d = np.arange(min(bins)*0.96, max(bins)*1.04, 1)
    pdf_plot = distro.pdf(d, *params)
    plt.plot(d, pdf_plot*22, lw=3.5, color='r')
    
    # Visualizar función de densidad marginal
    plt.bar(bins, marginal, width=12)
    plt.title('Contraste: densidad marginal vs. modelo de mejor ajuste')
    plt.xlabel('Potencia [MW]')
    plt.ylabel('Densidad Probabilística')
    plt.show()
    
    return marginal

#### 3.1.5. - Calculando el consumo diario y parámetros relevantes con `energia_diaria`

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

def energia_diaria(archivo_json):
    '''Importa la base de datos completa y devuelve
    un vector con la energía diaria, en MWh.
    '''
    
    # Cargar el "DataFrame"
    df = pd.read_json(archivo_json) 

    # Convertir en un array de NumPy
    datos = np.array(df)  

    # Crear vector con todos los valores horarios de demanda
    demanda = []

    # Extraer la magnitud de la demanda para todas las horas
    for hora in range(len(datos)):
        instante = datetime.fromisoformat(datos[hora][0]['fechaHora'])
        demanda.append(datos[hora][0]['MW'])

    # Separar las magnitudes en grupos de 24 (24 h)
    demanda = np.split(np.array(demanda), len(demanda) / 24)

    # Crear vector para almacenar la enegia a partir de la demanda
    energia = []

    # Calcular la energía diaria por la regla del trapecio
    for dia in range(len(demanda)):
        E = round(np.trapz(demanda[dia]), 2)
        energia.append(E)

    return energia

#### 3.1.6. - Determinación de parámetros del consumo diario con `parametros_energia`

In [ ]:
import numpy as np

def parametros_energia(vector_energia):
    '''Calcula los parámetros media y desviación
    estándar en el vector de energía.
    '''

    media = np.mean(vector_energia)
    desviacion = np.std(vector_energia)

    return media, desviacion

### 3.2. - Probando las funciones implementadas

Ahora bien, se puede elaborar una breve demostración práctica basada en las funciones anteriormente implementadas, haciendo sus respectivas llamadas:

In [ ]:
# Se eligen las dos horas que desean estudiarse
hora_1 = extraer_datos('demanda_2019.json', 3) 
hora_2 = extraer_datos('demanda_2019.json', 11) 

# Se ejecuta el análisis bivariado
xyp, xy, xbins, ybins = distribucion_conjunta(hora_1, hora_2, bins = 10)

In [ ]:
# Se llama obtiene los parámetros de mejor ajuste para cada hora
dist, params = evaluar_modelos(hora_1)

# Nota: esta función es de más lenta ejecución

In [ ]:
# Se contrasta la densidad marginal para cada hora, según su eje
densidad_marginal(xy, xbins, dist, params, eje='x')

---
## 4. - Asignaciones del proyecto

El análisis se hará sobre dos horas distintas del día, elegidas de forma aleatoria: una dentro de horas punta y otra no. 

**Asignación de horas**

La siguiente función utiliza los dígitos numéricos del carné de cada estudiante para asignar las horas con las que se construirán la variables aleatoria $X$ y la variable aleatoria $Y$. 

**Ejemplo**: el carné B12345 utiliza los dígitos 12345 y obtiene las horas 19 y 7.

In [ ]:
import random

def horas_asignadas(digitos):
    '''Elige una hora A en periodo punta
    y una hora B de los otros periodos,
    con los dígitos del carné como "seed"
    '''
    
    random.seed(digitos)
    punta = [11, 12, 18, 19, 20]
    valle = [7, 8, 9, 10, 13, 14, 15, 16, 17]
    nocturno = [21, 22, 23, 0, 1, 2, 3, 4, 5, 6]
    otro = valle + nocturno
    HX = punta[random.randrange(0, len(punta))]
    HY = otro[random.randrange(0, len(otro))]
    horas = 'Hora X: {}, hora Y: {}'.format(HX, HY)
    return horas

In [ ]:
mis_horas = horas_asignadas(5646846)
print(mis_horas)

# Bonus: los "docstrings" y la función help()
help(horas_asignadas)

### 4.1. - Función distribución múltiple bivariada

Es posible analizar una variable aleatoria en relación con otra. Para empezar, se puede visualizar su comportamiento "conjunto".

* (10%) Visualice los datos de la variable aleatoria múltiple a las dos horas elegidas, en la forma de un histograma conjunto.
* (15%) Determine las funciones de densidad marginales $f_X(x)$ y $f_Y(y)$ a partir de los datos y utilizando un modelo de mejor ajuste.
* (10%) Determine los valores esperados $E[X]$ y $E[Y]$ de los datos.

En este proyecto no deduciremos la *función de densidad conjunta*, que en general necesita de otros métodos numéricos para encontrar una expresión algebraica.

In [ ]:
# 4.1. - Función distribución múltiple bivariada

### 4.2. - Correlación entre el consumo a distintas horas del día

¿Un alto consumo de potencia a la hora $X$ está correlacionado con un alto consumo de potencia a la hora $Y$ del mismo día? Esta pregunta puede abordarse con las métricas de correlación y covarianza entre dos variables aleatorias.

* (15%) Calcule la correlación entre $X$ y $Y$, $E[XY] = R_{XY}$. Para esto, puede utilizar los datos provistos y la versión discreta de la fórmula de correlación, dada por $$\begin{aligned} R_{XY} = m_{11} & = E[XY] \\ & = \sum_{x=-\infty}^{\infty} \sum_{y=-\infty}^{\infty} x y ~ p(x, y) \end{aligned}$$ donde $p(x, y)$ son las probabilidades asociadas a cada par $(x, y)$.
* (10%) Análisis: ¿están correlacionadas estas variables? Recordar que la prueba de correlación es $E[XY] = E[X]E[Y]$ (puede aplicar una tolerancia del 5%).
* (10%) Determine la covarianza y el índice de correlación de Pearson.
* (10%) Análisis: ¿qué implica este valor del índice de correlación de Pearson?

In [ ]:
# 4.2. - Correlación entre el consumo a distintas horas del día

### 4.3. - Análisis de consumo de energía semanal

Semana a semana hay un consumo total de **energía** en el país. Este valor cambia aleatoriamente, pero, ¿será posible modelar su distribución de probabilidad? Más aún, ¿es posible predecir su distribución utilizando el teorema del limíte central y el conocimiento del consumo *diario* de energía?

Si el consumo de energía de *cada día de la semana* es una variable aleatoria $E_L, E_K, \ldots, E_D$, entonces el consumo total semanal está dado por:

$$
S_E = E_L + E_K + E_M + E_J + E_V + E_S + E_D
$$

El teorema del límite central para la suma, en la versión estudiada, aplica para la suma de variables aleatorias **iid** (*independientes e idénticamente distribuidas*). Esta suposición **no** aplica para los datos estudiados dado que, por ejemplo, el consumo del lunes no es el mismo que el del domingo. Sin embargo, se hará la aproximación con unos parámetros únicos $\mu$ (la media) y $\sigma$ (la desviación estándar) para la energía consumida durante todos los días del año.

Conocidos estos parámetros y considerando "razonablemente grande" la cantidad de elementos en la suma $N = 7$, el teorema establece que

$$
Z = \frac{S_E - \mu_{S_E}}{\sigma_{S_E}} = \frac{S_N - N\mu}{\sigma \sqrt{N}}
$$

tendrá una distribución $Z \sim \mathcal{N}(0, 1)$, es decir, normal estandarizada.

* (20 %) Evalúe la hipótesis descrita. Comente los resultados.

Para esto puede crear un histograma con los resultados del consumo de energía *semanal*. Puede además calcular los parámetros $\mu$ (la media) y $\sigma$ (la desviación estándar) de la energía diaria consumida y estimar la distribución de $S_E$.

In [ ]:
# 4.3. - Análisis de consumo de energía semanal

---

### Universidad de Costa Rica
#### Facultad de Ingeniería
##### Escuela de Ingeniería Eléctrica

---